In [13]:
dataset = "exp_density"
import sys; sys.path.insert(0, '..')
from exp_density_polynomial import *
from osmm import OSMM
from osmm import AlgMode
import pickle
import datetime

In [2]:
num_trials = 1
max_num_rounds = 100
compare_with_cvxpy = True
compare_with_mosek = False

In [3]:
"""
Structure of the alg_props dictionary is:
  <algorithm friendly name>: [<algorithm index>,
                              <number of rounds to run algorthm for (deprecated)>,
                              <algorithm update mode>]
"""
alg_props = {
        "r=20, M=20": [0, -1, AlgMode.LowRankQNBundle, 20, 20],
}
    
num_algs = len(alg_props)
solver = "MOSEK"

In [4]:
# Define structs:
objfs = np.zeros((num_trials, num_algs, max_num_rounds))
objfs_validation = np.zeros((num_trials, num_algs, max_num_rounds))
runtimes = np.zeros((num_trials, num_algs, max_num_rounds))
iters_takens = np.zeros((num_trials, num_algs))
damping_facs = np.ones((num_trials, num_algs, max_num_rounds))
mus = np.ones((num_trials, num_algs, max_num_rounds))
ts = np.ones((num_trials, num_algs, max_num_rounds))
Xs = np.zeros((n, num_trials, num_algs, max_num_rounds))
lower_bounds = np.ones((num_trials, num_algs, max_num_rounds)) * (-np.inf)
best_xs = np.zeros((n, num_trials, num_algs))
v_norms = np.zeros((num_trials, num_algs, max_num_rounds))
num_f_evas_line_searchs = np.zeros((num_trials, num_algs, max_num_rounds))
opt_res_norms = np.zeros((num_trials, num_algs, max_num_rounds))
q_norms = np.zeros((num_trials, num_algs, max_num_rounds))
f_grad_norms = np.zeros((num_trials, num_algs, max_num_rounds))
time_cost_details = np.zeros((6, num_trials, num_algs, max_num_rounds))

objfs_opt_cvxpy = np.zeros((num_trials,num_algs))
objfs_opt_mosek = np.zeros((num_trials,num_algs))
baseline_total_time_cost = np.zeros(num_trials)
baseline_mosek_solve_time = np.zeros(num_trials)

In [14]:
for trial_idx in range(num_trials):
    print("Trial %d/%d (dataset = %s):" % (trial_idx, num_trials - 1, dataset))
    W = generate_random_data()
    W_validation = generate_random_data()
    print("End generate data")
    if compare_with_mosek:
        t0 = time.time()
        x_opt_mosek, objf_opt = get_baseline_soln_mosek(W)
        baseline_mosek_solve_time[trial_idx] = time.time() - t0
        baseline_total_time_cost[trial_idx] = baseline_mosek_solve_time[trial_idx]
        print("opt objf value", objf_opt, "mosek total time", baseline_mosek_solve_time[trial_idx])
        objfs_opt_mosek[trial_idx, :] = objf_opt * np.ones(num_algs)
    else:
        objf_opt = None

    if compare_with_cvxpy:
        t0 = time.time()
        x_opt, objf_opt, baseline_mosek_solve_time[trial_idx] = get_baseline_soln_cvxpy(W, compare_with_all=False)
        baseline_total_time_cost[trial_idx] = time.time() - t0
        print("opt objf value", objf_opt, "CVXPY baseline total time", baseline_total_time_cost[trial_idx], 
              "CVXPY baseline solve time", baseline_mosek_solve_time[trial_idx])
        objfs_opt_cvxpy[trial_idx, :] = objf_opt * np.ones(num_algs)
        
    osmm_prob = OSMM(my_objf_torch, get_cvxpy_description, get_initial_val, W, W_validation, save_timing=True, store_x_all_iters=True)

    for alg_name in alg_props:
        print("    " + alg_name + ":")
        alg_idx = alg_props[alg_name][0]
        alg_mode = alg_props[alg_name][2]
        H_rank = alg_props[alg_name][3]
        M = alg_props[alg_name][4]
        
        osmm_prob.solve(max_num_rounds, alg_mode, r=H_rank, M=M, solver=solver)

        Xs[:, trial_idx, alg_idx, :] = osmm_prob.method_results["X_iters"]
        objfs[trial_idx, alg_idx, :] = osmm_prob.method_results["objf_iters"]
        objfs_validation[trial_idx, alg_idx, :] = osmm_prob.method_results["objf_validation_iters"]
        damping_facs[trial_idx, alg_idx, :] = osmm_prob.method_results["lambd_iters"]
        mus[trial_idx, alg_idx, :] = osmm_prob.method_results["mu_iters"]
        ts[trial_idx, alg_idx, :] = osmm_prob.method_results["t_iters"]
        lower_bounds[trial_idx, alg_idx, :] = osmm_prob.method_results["lower_bound_iters"]
        runtimes[trial_idx, alg_idx, :] = osmm_prob.method_results["runtime_iters"]
        best_xs[:, trial_idx, alg_idx] = osmm_prob.method_results["x_best"]
        v_norms[trial_idx, alg_idx, :] = osmm_prob.method_results["v_norm_iters"]
        num_f_evas_line_searchs[trial_idx, alg_idx, :] = osmm_prob.method_results["num_f_evas_line_search_iters"]
        opt_res_norms[trial_idx, alg_idx, :] = osmm_prob.method_results["opt_res_iters"]
        f_grad_norms[trial_idx, alg_idx, :] = osmm_prob.method_results["f_grad_norm_iters"]
        q_norms[trial_idx, alg_idx, :] = osmm_prob.method_results["q_norm_iters"]
        time_cost_details[:, trial_idx, alg_idx, :] = osmm_prob.method_results["time_cost_detail_iters"]
        iters_taken = osmm_prob.method_results["iters_taken"]
        print("")

Trial 0/0 (dataset = exp_density):
N_0 =  10000 N= 9781
N_0 =  10000 N= 9766
End generate data
Start to solve baseline problem by MOSEK
  MOSEK + CVXPY time cost  1.0081400871276855
  MOSEK solver time cost 0.7316279411315918
  Setup time cost None
  Objective value 0.5513278376866813
  Solver status  optimal.

opt objf value 0.5513278376866813 CVXPY baseline total time 1.0093529224395752 CVXPY baseline solve time 0.7316279411315918
    r=20, M=20:
Hutchinson #iters 41 rel. incr. 0.0009426630229483095 est. tr. 1.6800530866787928
lam_0 0.12000379190562806 Hutchinson time cost,  0.03798484802246094
iter= 10 objf_k+1= 0.5523053791060836 L_k+1= 0.545892453749671 lam_k+1= 0.002231784198563885 tk= 1.0 mu_k+1 0.7256313856000004 ||G_k+1||_F= 0.9279980964992036 tau_k+1 0.0030756445253790903
      Time elapsed (secs): 0.352150.




In [15]:
print("W shape = ", W.shape)
_, N = W.shape
print("log 10 N, n = ", np.log10(N), n)

W shape =  (15, 9781)
log 10 N, n =  3.9903832589062334 14


In [16]:
if not compare_with_cvxpy and not compare_with_mosek:
    baseline_objf_opt_s = None
elif not compare_with_mosek:
    baseline_objf_opt_s = objfs_opt_cvxpy
else:
    baseline_objf_opt_s = objfs_opt_mosek
print(baseline_objf_opt_s)

[[0.55132784]]


In [18]:
stuff = {}
stuff["objfs"] = objfs
stuff["objfs_validation"] = objfs_validation
stuff["lower_bounds"] = lower_bounds
stuff["runtimes"] = runtimes
stuff["opt_res_norms"] = opt_res_norms
stuff["f_grad_norms"] = f_grad_norms
stuff["q_norms"] = q_norms
stuff["damping_facs"] = damping_facs
stuff["ts"] = ts
stuff["Xs"] = Xs
stuff["best_xs"] = best_xs
stuff["v_norms"] = v_norms
stuff["num_f_evas_line_searchs"] = num_f_evas_line_searchs
stuff["iters_taken"] = iters_taken

stuff["time_cost_details"] = time_cost_details

stuff["alg_props"] = alg_props
stuff["baseline_total_time_cost"] = baseline_total_time_cost
stuff["baseline_mosek_solve_time"] = baseline_mosek_solve_time
stuff["baseline_objf_opt_s"] = baseline_objf_opt_s
now = datetime.datetime.now()
mmddyyhhmm = ("%d_%d_%d_%d_%d" % (now.month, now.day, now.year, now.hour, now.minute))
part_of_out_fn = dataset + "_pnt1_" + mmddyyhhmm
# pickle.dump(stuff, open("data_%s.pkl" % (part_of_out_fn), "wb"))